# Draft analysis 

---

Group name: mn086

---


## Introduction

*This section includes an introduction to the project motivation, data, and research question. Include a data dictionary* 

## Setup

In [1]:
import os
import pandas as pd

## Data

## Import data

In [2]:
root_raw = os.path.join('..', 'data', 'raw')
root_interim = os.path.join('..', 'data', 'interim')

data_veh = '46251-0021_de_2020_flat.csv' # PKW Zulassungszahlen
data_pop = '12211-Z-03_flat.csv'
data_income = 'AI-S-01_flat.csv'
data_crash = 'AI013-3_flat.csv'

df_veh = pd.read_csv(os.path.join(root_raw, data_veh), sep=";", decimal=',' )

In [3]:
df_veh = df_veh[['1_variable_attribute_code', '1_variable_attribute_label', '2_variable_attribute_code', '3_variable_attribute_code', 'value']]

### Data structure

In [4]:
df_veh

,1_variable_attribute_code,1_variable_attribute_label,2_variable_attribute_code,3_variable_attribute_code,value
0,9273,"Kelheim, Landkreis",KS-PIH,PKW-EURO6D,2
1,7231,"Bernkastel-Wittlich, Landkreis",KS-ELEKTRO,PKW-EURO6DT,-
2,12063,"Havelland, Landkreis",KS-HYBRID,PKW-EURO5,281
3,6438,"Offenbach, Landkreis",KS-SONST,PKW-EURO6-R,-
4,14523,Vogtlandkreis,KS-GAS,PKW-EURO5,136
...,...,...,...,...,...
38075,16076,"Greiz, Landkreis",KS-ELEKTRO,PKW-EURO6DT,-
38076,8316,"Emmendingen, Landkreis",NaN,PKW-EURO4,29478
38077,14625,"Bautzen, Landkreis",NaN,PKW-EURO3,13911
38078,7138,"Neuwied, Landkreis",KS-ELEKTRO,PKW-EURO3,-


In [5]:
df_veh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38080 entries, 0 to 38079
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   1_variable_attribute_code   38080 non-null  int64 
 1   1_variable_attribute_label  38080 non-null  object
 2   2_variable_attribute_code   33320 non-null  object
 3   3_variable_attribute_code   34272 non-null  object
 4   value                       38080 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.5+ MB


### Data corrections

In [6]:
df_veh.dropna(inplace=True)

In [7]:
# Spaltennamen neu definieren
df_veh.rename(columns= {
    '1_variable_attribute_code' : 'landkreis_id',
    '1_variable_attribute_label': 'landkreis',
    '2_variable_attribute_code': 'antrieb',
    '3_variable_attribute_code': 'emissionsgruppen',
    'value': 'anzahl'
    }, inplace=True)

In [8]:
df_veh['anzahl']= pd.to_numeric(df_veh['anzahl'], errors='coerce').fillna(0).astype(int)

In [9]:
df_veh.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29988 entries, 0 to 38079
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   landkreis_id      29988 non-null  int64 
 1   landkreis         29988 non-null  object
 2   antrieb           29988 non-null  object
 3   emissionsgruppen  29988 non-null  object
 4   anzahl            29988 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 1.4+ MB


In [10]:
# Attributausprägung für kategoriale Attribute anzeigen
for col in df_veh[['antrieb', 'emissionsgruppen']].columns:
    unique_values = df_veh[col].unique()
    print(f"Spalte '{col}' hat folgende Attributausprägungen:")
    print(unique_values)
    print("-" * 40)

Spalte 'antrieb' hat folgende Attributausprägungen:
['KS-PIH' 'KS-ELEKTRO' 'KS-HYBRID' 'KS-SONST' 'KS-GAS' 'KS-DIESEL'
 'KS-BENZIN']
----------------------------------------
Spalte 'emissionsgruppen' hat folgende Attributausprägungen:
['PKW-EURO6D' 'PKW-EURO6DT' 'PKW-EURO5' 'PKW-EURO6-R' 'PKW-EURO1'
 'PKW-EURO2' 'PKW-EURO3' 'PKW-SONST' 'PKW-EURO4']
----------------------------------------


In [11]:
# Landkreise mit einer Gebietsreform aus dem Datensatz löschen: diese enden beispielsweise mit "(bis 03.09.2011)"
# Filter: Zeilen löschen, bei denen 'landkreis' mit ')' gefolgt beliebig viele Leerzeichen (inklusive keine) endet
df_veh = df_veh[~df_veh['landkreis'].str.contains(r'\)\s*$', na=False)]

In [12]:
df_veh.to_csv(os.path.join(root_interim, 'veh.csv'))

In [13]:
# Dummy-Spalten aus kategorialen Atrributen erstellen
df_veh = pd.get_dummies(
    df_veh, 
    columns=['antrieb', 'emissionsgruppen'], 
    prefix={'antrieb': 'antrieb', 'emissionsgruppen': 'eg'}
)
df_veh.head()

,landkreis_id,landkreis,anzahl,antrieb_KS-BENZIN,antrieb_KS-DIESEL,antrieb_KS-ELEKTRO,antrieb_KS-GAS,antrieb_KS-HYBRID,antrieb_KS-PIH,antrieb_KS-SONST,eg_PKW-EURO1,eg_PKW-EURO2,eg_PKW-EURO3,eg_PKW-EURO4,eg_PKW-EURO5,eg_PKW-EURO6-R,eg_PKW-EURO6D,eg_PKW-EURO6DT,eg_PKW-SONST
0,9273,"Kelheim, Landkreis",2,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False
1,7231,"Bernkastel-Wittlich, Landkreis",0,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False
2,12063,"Havelland, Landkreis",281,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False
3,6438,"Offenbach, Landkreis",0,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False
4,14523,Vogtlandkreis,136,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False


In [14]:
# Spaltennamen umwandeln
df_veh.columns = df_veh.columns.str.lower().str.replace('ks-', '', regex=False).str.replace('pkw-', '', regex=False)

In [15]:
df_veh.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25200 entries, 0 to 38079
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   landkreis_id     25200 non-null  int64 
 1   landkreis        25200 non-null  object
 2   anzahl           25200 non-null  int64 
 3   antrieb_benzin   25200 non-null  bool  
 4   antrieb_diesel   25200 non-null  bool  
 5   antrieb_elektro  25200 non-null  bool  
 6   antrieb_gas      25200 non-null  bool  
 7   antrieb_hybrid   25200 non-null  bool  
 8   antrieb_pih      25200 non-null  bool  
 9   antrieb_sonst    25200 non-null  bool  
 10  eg_euro1         25200 non-null  bool  
 11  eg_euro2         25200 non-null  bool  
 12  eg_euro3         25200 non-null  bool  
 13  eg_euro4         25200 non-null  bool  
 14  eg_euro5         25200 non-null  bool  
 15  eg_euro6-r       25200 non-null  bool  
 16  eg_euro6d        25200 non-null  bool  
 17  eg_euro6dt       25200 non-null  boo

### Variable lists

### Data splitting

## Analysis

### Descriptive statistics

### Exploratory data analysis

### Relationships

## Model

### Select model

### Training and validation

### Fit model

### Evaluation on test set

### Save model



Save your model in the folder `models/`. Use a meaningful name and a timestamp.

## Conclusions